In [ ]:
import zipfile
import io
import urllib.request
import json

access_url = urllib.request.urlopen('http://data.assemblee-nationale.fr/static/openData/repository/15/loi/scrutins/Scrutins_XV.json.zip')

data_dict=dict()

z = zipfile.ZipFile(io.BytesIO(access_url.read()))
n_scrutin = len(z.infolist())


for i in range(n_scrutin):
    data = json.loads(z.read(z.infolist()[i]).decode())
    data_dict[i] = data


In [ ]:
import pandas as pd
from tqdm.notebook import tqdm

df_votants = pd.DataFrame(columns=['idScrutin', 'idVotant', 'vote', 'organeRefGroupe'])

for idScrutin, scrutin in tqdm(data_dict.items()):
    example = scrutin['scrutin']
    ventilation_dict = example['ventilationVotes']['organe']

    organeRefTot= ventilation_dict['organeRef']
    groupe_lists = ventilation_dict['groupes']['groupe']

for groupe_dict in groupe_lists:
    organe_ref_groupe = groupe_dict['organeRef']
    n_groupe = groupe_dict['nombreMembresGroupe']
    vote_dict = groupe_dict['vote']

    decompteVoix_groupe_dict = vote_dict['decompteVoix']
    decompteNominatif_groupe_dict = vote_dict['decompteNominatif']

    liste_non_votant_groupe = decompteNominatif_groupe_dict['nonVotants']
    liste_pours_groupe = decompteNominatif_groupe_dict['pours']
    liste_contres_groupe = decompteNominatif_groupe_dict['contres']

    if not liste_non_votant_groupe is None:
        if type(liste_non_votant_groupe['votant']) == dict:
            df_votants = df_votants.append({'idScrutin': idScrutin, 'idVotant': liste_non_votant_groupe['votant']['acteurRef'], 'vote' : 'Pour', 'organeRefGroupe' : organe_ref_groupe}, ignore_index=True)
        else :
            for vote in liste_non_votant_groupe['votant']:
                df_votants = df_votants.append({'idScrutin': idScrutin, 'idVotant' : vote['acteurRef'], 'vote' : 'Non-votant', 'organeRefGroupe' : organe_ref_groupe}, ignore_index=True)

    if not liste_pours_groupe is None :
        if type(liste_pours_groupe['votant']) == dict:
            df_votants = df_votants.append({'idScrutin': idScrutin, 'idVotant':liste_pours_groupe['votant']['acteurRef'], 'vote' : 'Pour', 'organeRefGroupe' : organe_ref_groupe}, ignore_index=True)
        else :
            for vote in liste_pours_groupe['votant']:
                df_votants = df_votants.append({'idScrutin': idScrutin, 'idVotant' : vote['acteurRef'], 'vote' : 'Pour', 'organeRefGroupe' : organe_ref_groupe}, ignore_index=True)

    if not liste_contres_groupe is None:
        if type(liste_contres_groupe['votant']) == dict:
            df_votants = df_votants.append({'idScrutin': idScrutin, 'idVotant':liste_contres_groupe['votant']['acteurRef'], 'vote' : 'Contre', 'organeRefGroupe' : organe_ref_groupe}, ignore_index=True)
        else :
            for vote in liste_contres_groupe['votant']:
                df_votants = df_votants.append({'idScrutin': idScrutin, 'idVotant' : vote['acteurRef'], 'vote' : 'Contre', 'organeRefGroupe' : organe_ref_groupe}, ignore_index=True)

df_votants

,idScrutin,idVotant,vote,organeRefGroupe
0,0,PA606171,Non-votant,PO730964
1,0,PA721824,Non-votant,PO730964
2,0,PA719952,Pour,PO730964
3,0,PA607395,Pour,PO730964
4,0,PA718710,Pour,PO730964
...,...,...,...,...
308924,3116,PA335612,Pour,PO730940
308925,3116,PA607619,Contre,PO759900
308926,3116,PA719608,Pour,PO723569
308927,3116,PA721932,Pour,PO723569


In [ ]:
import pandas as pd
df_scrutin = pd.DataFrame(columns = ['id_scrutin', 'date_scrutin', 'code_type_vote', 'titre', 'demandeur', 'organe_ref', 'votants', 
                                     'pour', 'contre', 'non_votants', 'non_votants_volontaires', 'abstention', 'resultat'])

for idScrutin, scrutin in data_dict.items():
    scrutin = scrutin['scrutin']
    df_scrutin = df_scrutin.append({'id_scrutin' : idScrutin, 'date_scrutin' : scrutin['dateScrutin'], 'code_type_vote': scrutin['typeVote']['codeTypeVote'],
                  'titre' : scrutin['titre'], 'demandeur' : scrutin['demandeur']['texte'], 'organe_ref' : scrutin['organeRef'],
                  'votants': scrutin['syntheseVote']['nombreVotants'], 'pour' : scrutin['syntheseVote']['decompte']['pour'], 
                  'contre' : scrutin['syntheseVote']['decompte']['contre'], 'non_votants' : scrutin['syntheseVote']['decompte']['nonVotants'],
                  'non_votants_volontaires' : scrutin['syntheseVote']['decompte']['nonVotantsVolontaires'],
                  'abstention': scrutin['syntheseVote']['decompte']['abstentions'],'resultat': scrutin['sort']['code']}, ignore_index=True)

df_scrutin

In [ ]:
df_deputes = pd.DataFrame(columns = ['id_scrutin', 'date_scrutin', 'code_type_vote', 'titre', 'demandeur', 'organe_ref', 'votants', 
                                     'pour', 'contre', 'non_votants', 'non_votants_volontaires', 'abstention', 'resultat'])

for idScrutin, scrutin in data_dict.items():
  scrutin = scrutin['scrutin']
  df_deputes = df_deputes.append({'id_scrutin' : idScrutin, 'date_scrutin' : scrutin['dateScrutin'], 'code_type_vote': scrutin['typeVote']['codeTypeVote'],
                  'titre' : scrutin['titre'], 'demandeur' : scrutin['demandeur']['texte'], 'organe_ref' : scrutin['organeRef'],
                  'votants': scrutin['syntheseVote']['nombreVotants'], 'pour' : scrutin['syntheseVote']['decompte']['pour'], 
                  'contre' : scrutin['syntheseVote']['decompte']['contre'], 'non_votants' : scrutin['syntheseVote']['decompte']['nonVotants'],
                  'non_votants_volontaires' : scrutin['syntheseVote']['decompte']['nonVotantsVolontaires'],
                  'abstention': scrutin['syntheseVote']['decompte']['abstentions'],'resultat': scrutin['sort']['code']}, ignore_index=True)

df_deputes